In [38]:
import polars as pl
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
df=[]
for sheet in ['159920 CH Equity', 'HSI Index', 'HIJ5 Index']:
    tmp = pl.read_excel('Jerry_three index.xlsx', sheet_name=sheet)
    tmp = tmp.with_columns(
        pl.col('Last Price').alias(sheet),
    )
    df.append(tmp.select(['Date', sheet]))

df = df[0].join(
    df[1],
    on='Date',
    how='outer',
    coalesce=True
).join(
    df[2],
    on='Date',
    how='outer',
    coalesce=True
)

# convet Date from New york time to Hong Kong time
df = df.with_columns(
    pl.col('Date').dt.replace_time_zone('America/New_York').dt.convert_time_zone('Asia/Hong_Kong')
)

# Melt the DataFrame for easier plotting with Plotly
df = df.sort('Date', descending=False)


/var/folders/1f/5ggn6k991fv0z5vzr6xpyxx80000gn/T/ipykernel_18447/3505246628.py:9: DeprecationWarning:

Use of `how='outer'` should be replaced with `how='full'`.

/var/folders/1f/5ggn6k991fv0z5vzr6xpyxx80000gn/T/ipykernel_18447/3505246628.py:14: DeprecationWarning:

Use of `how='outer'` should be replaced with `how='full'`.

/var/folders/1f/5ggn6k991fv0z5vzr6xpyxx80000gn/T/ipykernel_18447/3505246628.py:28: DeprecationWarning:

`DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`



In [70]:
filtered_df = df.filter(
    pl.col("Date") >= pl.datetime(2025, 4, 7, 9, 30, time_unit="ms").dt.replace_time_zone("Asia/Hong_Kong")
).with_columns(
    pl.col('159920 CH Equity')*15912.7465
)
import plotly.graph_objects as go

fig = go.Figure()

# Add traces for each index
for column in filtered_df.columns[1:]:
    fig.add_trace(go.Scatter(
        x=filtered_df['Date'],
        y=filtered_df[column],
        mode='lines',
        name=column
    ))

# Add a line for 159920 - HIJ5 and use secondary y-axis
fig.add_trace(go.Scatter(
    x=filtered_df['Date'],
    y=(filtered_df['159920 CH Equity'] - filtered_df['HIJ5 Index']),
    mode='lines',
    name='159920 - HIJ5',
    yaxis='y2'
))

# Add a line for 159920 - HIJ5 and use secondary y-axis
fig.add_trace(go.Scatter(
    x=filtered_df['Date'],
    y=(filtered_df['159920 CH Equity'] - filtered_df['HSI Index']),
    mode='lines',
    name='159920 - HSI',
    yaxis='y2'
))

# Add a line for 159920 - HIJ5 and use secondary y-axis
fig.add_trace(go.Scatter(
    x=filtered_df['Date'],
    y=(filtered_df['HSI Index'] - filtered_df['HIJ5 Index']),
    mode='lines',
    name='HSI - HIJ5',
    yaxis='y2'
))

# Update layout
fig.update_layout(
    title="Filtered Index Trends Over Time",
    xaxis_title="Date",
    yaxis_title="Value",
    yaxis2=dict(
        title="Difference (159920 - HIJ5)",
        overlaying='y',
        side='right'
    ),
    legend_title="Index",
    height=800,
    width=1400,
)
# add a bar under the line chart to adjust the time range
fig.update_xaxes(
    rangeslider_visible=True,
    )
fig.show()